In [24]:
#Essentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv 
import datetime as dt

#For scraping Reddit
import praw
from psaw import PushshiftAPI

#For EDA
import re
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.cluster import KMeans
import sklearn.cluster
import sklearn.decomposition
import matplotlib.colors
from textblob import TextBlob
from gensim.models import Word2Vec
import nltk
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textstat
import gensim.corpora as corpora
from pprint import pprint
import gensim
from gensim.utils import simple_preprocess
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os

# Scraping data from reddit

In [2]:
# Before we can use praw, we have to create a Reddit instance do identify 
reddit = praw.Reddit(client_id='yyW8xfXOePfLhAE3sGhxXg', client_secret='ki5wj55ciBeoKhQghRtEJKS8QKJjPw', 
                     user_agent='Scraping')


# Mental Heatlh

In [187]:
#Thanks to dmarx for the clear code on pulling from PSAW! PSAW docs are surprisingly unclear.
#https://github.com/dmarx/psaw
api = PushshiftAPI()

#Start date we're beginning to pull posts from
start_epoch=int(dt.datetime(2015, 1, 1).timestamp())


#Pulling submissions from mentalhealth (both the title and body of a post)

gen_subMH = list(api.search_submissions(after=start_epoch,
                            subreddit='mentalhealth',
                            filter=['title', 'id','score','link_flair_text','selftext'],
                            limit=100))


#Converting to dataframe
MHsub = pd.DataFrame([thing.d_ for thing in gen_subMH])


C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [188]:
MHsub.head()

,created_utc,id,score,selftext,title,created,link_flair_text
0,1663442751,xgv9ri,1,I (22m) realized I do not know what genuine ha...,"Please, I need advice",1.663457e+09,NaN
1,1663442607,xgv7re,1,"For some context, I also have ADHD. I grew up ...",I can’t handle my partners depression and ADD ...,1.663457e+09,Venting
2,1663442548,xgv6y3,1,"I'm a stereotypically depressed 17 year old, I...",everything is just continually bland,1.663457e+09,Opinion / Thoughts
3,1663442532,xgv6qa,1,I'm tired. I'm struggling and I'm tired.\n\nI'...,struggling with mental health,1.663457e+09,Venting
4,1663442085,xgv07m,1,"Sh(s3lf harm) \n\nOkay, my thighs are easy to ...",How do I hide sh?,1.663456e+09,Question


In [191]:
MHids = list(MHsub.id)

In [197]:
new_idsMH = ['t3_' + i for i in MHids]

In [198]:
#Pulling comments from relationship_advice

gen_comMH = api.search_comments(after=start_epoch, 
                            subreddit='mentalhealth',
                            filter=['body', 'link_id', 'id', 'score','submission'],
                            link_id = new_idsMH,
                            limit=350)

MHcom = pd.DataFrame([thing.d_ for thing in gen_comMH])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [199]:
MHcom

,body,created_utc,id,link_id,score,created
0,Are you okay now ? A job dealing with economy ...,1663443565,iou27bo,t3_xgu293,1,1.663458e+09
1,If you are struggling with motivation like me:...,1663443520,iou23ki,t3_xgumfz,1,1.663458e+09
2,"Damn, I feel you. That's how I feel.",1663443462,iou1ylv,t3_xgusaw,1,1.663458e+09
3,Yeah I just always wear long sleeve shirts mys...,1663443148,iou1897,t3_xgv07m,1,1.663458e+09
4,Im gonna kink shame you bud. We've all got stu...,1663443046,iou0zun,t3_xgpq15,1,1.663457e+09
...,...,...,...,...,...,...
247,ive been seeing therapists for almost ten year...,1663415650,iosbs2z,t3_xgkb2s,1,1.663430e+09
248,I've been seeing psychiatrists for 25 years wi...,1663415578,iosbo5i,t3_xgkb2s,1,1.663430e+09
249,there is no inner strength. there is no inner ...,1663415515,iosbkht,t3_xgkb2s,1,1.663430e+09
250,You need to tap in to your inner strength and ...,1663415164,iosb0st,t3_xgkb2s,1,1.663430e+09


# Mental Illness subreddit

In [195]:
gen_subMI = list(api.search_submissions(after=start_epoch,
                            subreddit='mentalillness',
                            filter=['title', 'id','score','link_flair_text','selftext'],
                            limit=100))


#Converting to dataframe
MIsub = pd.DataFrame([thing.d_ for thing in gen_subMI])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [196]:
MIsub

,created_utc,id,link_flair_text,score,selftext,title,created
0,1663443535,xgvkr9,Relationships,1,heavy mention of killing . breif mention of ed...,worried about my boyfriend,1.663458e+09
1,1663438740,xgtoep,NaN,1,"this has happened for a while, i seriously don...",sometimes i feel i've became a different person,1.663453e+09
2,1663438416,xgtjlr,NaN,1,,dissociation *please read*,1.663453e+09
3,1663437352,xgt49i,NaN,1,[removed],I'm not sick but I'm not well,1.663452e+09
4,1663435374,xgsbmp,Advice Needed,1,,I really struggle with putting my problems int...,1.663450e+09
...,...,...,...,...,...,...,...
95,1663209714,xekxby,NaN,1,,what are some mental disorders that cause sudd...,1.663224e+09
96,1663207653,xek6ht,NaN,1,[removed],Sometimes I really wish I could either scream ...,1.663222e+09
97,1663206187,xejo26,NaN,1,My anxiety goes up and down throughout the wee...,Only feel anxiety,1.663221e+09
98,1663204914,xej81h,Medication,1,Been diagnosed with persistent depressive diso...,Overreacting,1.663219e+09


In [200]:
MIids = list(MIsub.id)

In [201]:
new_MIids = ['t3_' + i for i in MIids]

In [202]:
gen_comMI = api.search_comments(after=start_epoch, 
                            subreddit='mentalillness',
                            filter=['body', 'link_id', 'id', 'score','submission'],
                            link_id = new_MIids,
                            limit=350)

MIcom = pd.DataFrame([thing.d_ for thing in gen_comMI])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [203]:
MIcom

,body,created_utc,id,link_id,score,created
0,Sometimes medication or therapy is not effecti...,1663442940,iou0qv1,t3_xg8718,1,1.663457e+09
1,I understand what you're saying but how does t...,1663441917,iotycyt,t3_xg8718,1,1.663456e+09
2,So sir it means dr remains safe?\nMeans if a p...,1663441915,iotycuc,t3_xgj5q2,1,1.663456e+09
3,"Lol, great! I do hope you find the answers you...",1663441797,ioty2u7,t3_xgj5q2,1,1.663456e+09
4,Sorry i dont want to hate u,1663441737,iotxxm8,t3_xgj5q2,1,1.663456e+09
...,...,...,...,...,...,...
175,BPD (borderline personality disorder) one of t...,1663214032,iohlbb5,t3_xekxby,1,1.663228e+09
176,It appears you may be asking if you or a loved...,1663213830,iohkxxj,t3_xemc5k,1,1.663228e+09
177,I’ll take you up on your offer. I’m not a psyc...,1663212058,iohhkib,t3_xel70i,1,1.663226e+09
178,Most of them. Because being crazy stresses peo...,1663210125,iohdovk,t3_xekxby,1,1.663225e+09


In [ ]:
#Get rid of emojis, get rid of removed comments